# Farbricare ETL

## Step 1: Load SQL and create database

In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
%sql sqlite:///FabricareData.db
conn = sqlite3.connect('FabricareData.db')

## Step 2: Create tables as designed in ERD

In [3]:
%%sql
DROP TABLE IF EXISTS ACCOUNT;
CREATE TABLE ACCOUNT (
    Account_id INTEGER PRIMARY KEY,
    Item INTEGER NOT NULL,
    Date TEXT NOT NULL,
    Invoice TEXT NOT NULL,
    MasterAccountNo INTEGER NOT NULL,
    AccountNo INTEGER NOT NULL,
    Amount REAL,
    Payment REAL,
    Balance REAL,
    Type TEXT    
);

 * sqlite:///FabricareData.db
Done.
Done.


[]

In [4]:
%%sql
DROP TABLE IF EXISTS GARMENTLABEL;
CREATE TABLE GARMENTLABEL (
    GarmentLabel_id INTEGER PRIMARY KEY,
    Item INTEGER NOT NULL,
    Barcode INTEGER NOT NULL,
    AccountNo INTEGER,
    FirstDate TEXT NOT NULL,
    LastDate TEXT NOT NULL,
    Count INTEGER NOT NULL,
    GarmentType INTEGER NOT NULL,
    BrandName TEXT NOT NULL,
    Color1 INTEGER,
    Upcharge1 INTEGER,
    Color2 INTEGER,
    UpCharge2 INTEGER,
    Color3 INTEGER,
    Upcharge3 INTEGER,
    Color4 INTEGER,
    Upcharge4 INTEGER,
    Date_of_Revenue TEXT,
    LastPrice REAL
);

 * sqlite:///FabricareData.db
Done.
Done.


[]

In [5]:
%%sql

DROP TABLE IF EXISTS CUSTOMER;
CREATE TABLE CUSTOMER (
    Customer_id INTEGER PRIMARY KEY,
    Item INTEGER NOT NULL,
    AccountNo INTEGER NOT NULL,
    Name TEXT NOT NULL,
    Address TEXT,
    City TEXT,
    State TEXT,
    Zip INTEGER,
    RouteNo TEXT,
    First_date TEXT NOT NULL,
    Last_date TEXT NOT NULL,
    Total_garment INTEGER,
    Total_ytd REAL,
    Total_last REAL,
    Credit REAL,
    Total_cred REAL,
    Wcredit REAL,
    Total_wcre REAL,
    Account INTEGER,
    Account_balance REAL,
    Account_30 REAL,
    Account_60 REAL,
    Account_90 REAL,
    Last_paid_date TEXT,
    Last_payment REAL,
    Pricegroup INTEGER NOT NULL,
    Phone INTEGER NOT NULL,
    TotalLauYtd REAL,
    TotalLauLY REAL,
    TotalLauPcs INTEGER,
    TotalOtherYtd REAL,
    TotalOtherLY REAL,
    TotalOtherPcs INTEGER
);

 * sqlite:///FabricareData.db
Done.
Done.


[]

In [6]:
%%sql

DROP TABLE IF EXISTS ROUTE;
CREATE TABLE ROUTE (
    Route_id INTEGER PRIMARY KEY,
    Item INTEGER NOT NULL,
    RouteNo TEXT NOT NULL,
    RouteName TEXT NOT NULL,
    Rmon INTEGER,
    Rtue INTEGER,
    Rwed INTEGER,
    Rthu INTEGER,
    Rfri INTEGER,
    Rsat INTEGER,
    Rsun INTEGER,
    PriceGroup INTEGER NOT NULL
);

 * sqlite:///FabricareData.db
Done.
Done.


[]

In [23]:
%%sql

DROP TABLE IF EXISTS ROUTE_PICKED;
CREATE TABLE ROUTE_PICKED (
    RoutePicked_id INTEGER PRIMARY KEY,
    Item INTEGER NOT NULL,
    AccountNo INTEGER NOT NULL,
    Picked INTEGER,
    PickDate TEXT NOT NULL,
    PickTime TEXT NOT NULL,
    PickEmp TEXT ,
    PickEmpNo INTEGER,
    StopNo INTEGER ,
    RouteNo 
);

 * sqlite:///FabricareData.db
Done.
Done.


[]

## Step 3: Import CSV files

In [8]:
# importing GarmLabel csv
data = pd.read_csv('./Source Data/GarmLabel_cleaned.csv')   # path to directory
data.to_sql('Import_GarmLabel_cleaned',conn,if_exists='append',index=False) # converting to sql & unique

In [9]:
# importing Account csv
data = pd.read_csv('./Source Data/Account.csv')   # path to directory
data.to_sql('Import_Account',conn,if_exists='append',index=False) # converting to sql & unique

In [10]:
# importing Customer csv
data = pd.read_csv('./Source Data/Customer.csv')   # path to directory
data.to_sql('Import_Customer',conn,if_exists='append',index=False) # converting to sql & unique

In [11]:
# importing Route csv
data = pd.read_csv('./Source Data/Route.csv')   # path to directory
data.to_sql('Import_Route',conn,if_exists='append',index=False) # converting to sql & unique

/opt/tljh/user/lib/python3.6/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [12]:
# importing Route csv
data = pd.read_csv('./Source Data/RoutePicked.csv')   # path to directory
data.to_sql('Import_RoutePicked',conn,if_exists='append',index=False) # converting to sql & unique

In [13]:
%%sql
--check for duplication in GarmLabel
SELECT 
    (SELECT Count(*) FROM IMPORT_GarmLabel_cleaned) as 'Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_GarmLabel_cleaned)) as 'DistinctCount';

 * sqlite:///FabricareData.db
Done.


Count,DistinctCount
485588,485588


In [14]:
%%sql
-- check for duplication in Account

SELECT 
    (SELECT Count(*) FROM IMPORT_Account) as 'Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_Account)) as 'DistinctCount';

 * sqlite:///FabricareData.db
Done.


Count,DistinctCount
694648,694648


In [15]:
%%sql
--check for duplication in Customer

SELECT 
    (SELECT Count(*) FROM IMPORT_Customer) as 'Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_Customer)) as 'DistinctCount';

 * sqlite:///FabricareData.db
Done.


Count,DistinctCount
10337,10337


In [16]:
%%sql
--check for duplication in Route

SELECT 
    (SELECT Count(*) FROM IMPORT_Route) as 'Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_Route)) as 'DistinctCount';

 * sqlite:///FabricareData.db
Done.


Count,DistinctCount
24,24


In [17]:
%%sql
--check for duplication in RoutePicked

SELECT 
    (SELECT Count(*) FROM IMPORT_RoutePicked) as 'Count',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_RoutePicked)) as 'DistinctCount';

 * sqlite:///FabricareData.db
Done.


Count,DistinctCount
433566,433566


## Step 4: Populating the Tables

In [18]:
%%sql
DELETE FROM ACCOUNT;
INSERT INTO ACCOUNT(Item,Date,Invoice,MasterAccountNo,AccountNo,Amount,Payment,Balance,Type)
    SELECT DISTINCT Item,Date,Invoice,MasterAccountNo,AccountNo,Amount,Payment,Balance,Type
    FROM Import_Account;
SELECT * FROM ACCOUNT LIMIT 5;

 * sqlite:///FabricareData.db
0 rows affected.
694648 rows affected.
Done.


Account_id,Item,Date,Invoice,MasterAccountNo,AccountNo,Amount,Payment,Balance,Type
1,220996,1/1/2015,Invoice # A2984409,8691876,8691876,27.75,0.0,78.29,D
2,304049,1/1/2015,Invoice # D2983364,2760597,2760597,42.81,0.0,42.81,D
3,304173,1/1/2015,Invoice # D2983371,2499686,2499686,42.18,0.0,59.04,D
4,304212,1/1/2015,Invoice # D2983372,2760597,2760597,67.38,0.0,110.19,D
5,304394,1/1/2015,Invoice # D2983382,5699323,5699323,8.4,0.0,8.4,L


In [19]:
%%sql
DELETE FROM GARMENTLABEL;
INSERT INTO GARMENTLABEL(Item,Barcode,AccountNo,FirstDate,LastDate,Count,GarmentType,BrandName,Color1,Upcharge1,Color2,UpCharge2,Color3,Upcharge3,Color4,Upcharge4,Date_of_Revenue,LastPrice)
    SELECT DISTINCT Item,Barcode,AccountNo,FirstDate,LastDate,Count,GarmentType,BrandName,Color1,Upcharge1,Color2,UpCharge2,Color3,Upcharge3,Color4,Upcharge4,Date_of_Revenue,LastPrice
    FROM Import_GarmLabel_cleaned
    ORDER BY FirstDate;
SELECT * FROM GARMENTLABEL LIMIT 5;

 * sqlite:///FabricareData.db
0 rows affected.
485588 rows affected.
Done.


GarmentLabel_id,Item,Barcode,AccountNo,FirstDate,LastDate,Count,GarmentType,BrandName,Color1,Upcharge1,Color2,UpCharge2,Color3,Upcharge3,Color4,Upcharge4,Date_of_Revenue,LastPrice
1,40718,99493174,9726973,1/10/2006,10/9/2015,113,507,Kirkland,1191,0,1181,0,1205,0,0,0,10/9/2015,3.3
2,40643,3579294031,3579294,1/10/2006,10/31/2015,3,501,Old Navy,1190,0,0,0,0,0,0,0,10/31/2015,2.9
3,40773,6559976073,6559976,1/10/2006,1/5/2016,10,501,Brooks Brothers,1181,0,0,0,0,0,0,0,1/5/2016,2.95
4,40772,6551576025,6551576,1/10/2006,3/3/2017,8,507,Brooks Brothers,1181,0,0,0,0,0,0,0,3/3/2017,3.65
5,40794,9679220055,9679220,1/10/2006,4/17/2018,6,501,Brooks Brothers,1189,0,1206,0,0,0,0,0,4/17/2018,3.05


In [20]:
%%sql
DELETE FROM CUSTOMER;
INSERT INTO CUSTOMER(Item,AccountNo,Name,Address,City,State,Zip,RouteNo,First_date,Last_date,Total_garment,Total_ytd,Total_last,Credit,Total_cred,Wcredit,Total_wcre,Account,Account_balance,Account_30,Account_60,Account_90,Last_paid_date,Last_payment,Pricegroup,Phone,TotalLauYtd,TotalLauLY,TotalLauPcs,TotalOtherYtd,TotalOtherLY,TotalOtherPcs)
    SELECT DISTINCT Item,AccountNo,Name,Address,City,State,Zip,RouteNo,First_date,Last_date,Total_garment,Total_ytd,Total_last,Credit,Total_cred,Wcredit,Total_wcre,Account,Account_balance,Account_30,Account_60,Account_90,Last_paid_date,Last_payment,Pricegroup,Phone,TotalLauYtd,TotalLauLY,TotalLauPcs,TotalOtherYtd,TotalOtherLY,TotalOtherPcs
    FROM Import_Customer
    ORDER BY First_date;
SELECT * FROM CUSTOMER LIMIT 5;

 * sqlite:///FabricareData.db
0 rows affected.
10337 rows affected.
Done.


Customer_id,Item,AccountNo,Name,Address,City,State,Zip,RouteNo,First_date,Last_date,Total_garment,Total_ytd,Total_last,Credit,Total_cred,Wcredit,Total_wcre,Account,Account_balance,Account_30,Account_60,Account_90,Last_paid_date,Last_payment,Pricegroup,Phone,TotalLauYtd,TotalLauLY,TotalLauPcs,TotalOtherYtd,TotalOtherLY,TotalOtherPcs
1,26279,2561464,"Tappert, Judy",50 Brookband Rd,Fairfield,CT,6824,None,1/1/2008,7/24/2019,0,0.0,30.51,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0,None,None,30,256-1464,0.0,0.0,0,0.0,0.0,0
2,683,3271348,"Wardaki, Karim",24 Henry Street,Darien,CT,6820,None,1/10/1998,4/15/2018,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,None,0.0,1,327-1348,0.0,0.0,0,0.0,0.0,0
3,902,3482244,"HOYT, I",89 J,None,None,None,None,1/10/2001,5/3/2019,0,0.0,23.73,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,None,0.0,1,348-2244,0.0,0.0,0,0.0,0.0,0
4,12987,2548473,"Taikowski, Richard",360 GALLOPING HILL RD.,Fairfield,CT,6824,None,1/10/2002,3/3/2015,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,None,None,1,218-0301,0.0,0.0,0,0.0,0.0,0
5,12978,2548240,"Toner, Angela",107 Buckboard Lane,Fairfield,CT,6824,None,1/10/2002,1/13/2016,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,2/12/2016,9.65,1,254-8240,0.0,0.0,0,0.0,0.0,0


In [21]:
%%sql
DELETE FROM ROUTE;
INSERT INTO ROUTE(Item,RouteNo,RouteName,Rmon,Rtue,Rwed,Rthu,Rfri,Rsat,Rsun,PriceGroup)
    SELECT DISTINCT Item,RouteNo,RouteName,Rmon,Rtue,Rwed,Rthu,Rfri,Rsat,Rsun,PriceGroup
    FROM Import_Route
    ORDER BY RouteNo;
SELECT * FROM ROUTE LIMIT 5;

 * sqlite:///FabricareData.db
0 rows affected.
24 rows affected.
Done.


Route_id,Item,RouteNo,RouteName,Rmon,Rtue,Rwed,Rthu,Rfri,Rsat,Rsun,PriceGroup
1,20,2,Darien,1,0,0,1,0,0,0,2
2,17,27,Southport Tue/Fri,0,1,0,0,1,0,0,27
3,22,3,"Darien- Tuesday, Friday",0,1,0,0,1,0,0,3
4,8,30,Fairfield-Monday/Thursday,1,0,0,1,0,0,0,30
5,9,31,Westport-Tuesday/Friday,0,1,0,0,1,0,0,31


In [24]:
%%sql
DELETE FROM ROUTE_PICKED;
INSERT INTO ROUTE_PICKED(Item,AccountNo,Picked,PickDate,PickTime,PickEmp,PickEmpNo,StopNo,RouteNo)
    SELECT DISTINCT Item,AccountNo,Picked,PickDate,PickTime,PickEmp,PickEmpNo,StopNo,RouteNo
    FROM Import_RoutePicked
    ORDER BY PickDate;
SELECT * FROM ROUTE_PICKED LIMIT 5;

 * sqlite:///FabricareData.db
0 rows affected.
433566 rows affected.
Done.


RoutePicked_id,Item,AccountNo,Picked,PickDate,PickTime,PickEmp,PickEmpNo,StopNo,RouteNo
1,177,1001666,0,1/10/2017,11:04:34 AM,Mark,937,123,5
2,626,1002340,0,1/10/2017,10:48:35 AM,Mark,937,109,5
3,1922,1003925,1,1/10/2017,8:33:58 AM,Mark,937,141,3
4,3048,1012764,1,1/10/2017,2:57:14 PM,Christopher,727,206,35
5,3609,1013891,1,1/10/2017,8:29:40 AM,Christopher,727,108,35


## Step 5: Clear and Regain Storage

In [ ]:
%%sql
DELETE FROM IMPORT_CATALOG_COURSES;
DELETE FROM IMPORT_COURSE_OFFERINGS;
DELETE FROM IMPORT_COURSE_MEETINGS;

In [ ]:
%%sql
DROP TABLE IMPORT_CATALOG_COURSES;
DROP TABLE IMPORT_COURSE_OFFERINGS;
DROP TABLE IMPORT_COURSE_MEETINGS;

In [ ]:
%%sql
vacuum;